In [1]:
import sys
sys.dont_write_bytecode = True

# for debug print
def chkprint(*args):
    from inspect import currentframe
    names = {id(v):k for k,v in currentframe().f_back.f_locals.items()}
    print(', '.join(names.get(id(arg),'???')+' : '+str(type(arg))+' = '+repr(arg) for arg in args))

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import all_estimators
import warnings
import time

In [7]:
# 時間計測 start
start = time.time()

# ワイン白データ読み込み
wine = pd.read_csv("winequality-white.csv", sep=";", encoding="utf-8")

# データをラベルとデータに分離 
y = wine["quality"]
x = wine.drop("quality", axis=1)

# y のラベルを付け直す 
newlist = []
for v in list(y):
    if v <= 4:
        newlist += [0]
    elif v <=7:
        newlist += [1]
    else:
        newlist += [2]
y = newlist


# 学習用とテスト用に分離する 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

# classifierのアルゴリズム全てを取得する --- (※1)
allAlgorithms = all_estimators(type_filter="classifier")
warnings.simplefilter("error")

for(name, algorithm) in allAlgorithms :
  try :
    print("\n name       : " + name)
    #print("algorithm: " + str(algorithm))  
    chkprint(algorithm)
    
    # 各アリゴリズムのオブジェクトを作成 --- (※2)
    clf = algorithm()

    # 学習して、評価する --- (※3)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(name,"の正解率 = " , accuracy_score(y_test, y_pred))
  
  # WarningやExceptionの内容を表示する --- (※4)
  except Warning as w :
    print("\033[33m"+"Warning："+"\033[0m", name, ":", w.args)
  except Exception as e :
    print("\033[31m"+"Error："+"\033[0m", name, ":", e.args)
    
# 時間計測 finish
elapsed_time = time.time() - start
print ("\n elapsed_time:{0}".format(elapsed_time) + "[sec]")


 name       : AdaBoostClassifier
algorithm : <class 'abc.ABCMeta'> = <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>
Warning： AdaBoostClassifier : ('`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.\nDeprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations',)

 name       : BaggingClassifier
algorithm : <class 'abc.ABCMeta'> = <class 'sklearn.ensemble._bagging.BaggingClassifier'>
Warning： BaggingClassifier : ('`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar ty

SVC の正解率 =  0.9183673469387755

 name       : StackingClassifier
algorithm : <class 'abc.ABCMeta'> = <class 'sklearn.ensemble._stacking.StackingClassifier'>
Error： StackingClassifier : ("__init__() missing 1 required positional argument: 'estimators'",)

 name       : VotingClassifier
algorithm : <class 'abc.ABCMeta'> = <class 'sklearn.ensemble._voting.VotingClassifier'>
Error： VotingClassifier : ("__init__() missing 1 required positional argument: 'estimators'",)

 elapsed_time:55.14176392555237[sec]
